# 1. Simulation and Binding

This notebook runs the core agent-based simulation and performs hyperdimensional binding to record agent interactions. It serves as the first step in the Fluid-HD experiment pipeline.

**Workflow:**
1.  **Setup & Imports**: Load necessary libraries and add the `src` directory to the path.
2.  **Set Hyperparameters**: Define the parameters for this specific simulation run (e.g., N, D, T).
3.  **Instantiate Core Components**: Create instances of the `AgentSimulator`, `HDBinder`, and `MemoryStore`.
4.  **Run Simulation Loop**: Iterate through time steps, updating agent states, determining groups, and binding co-occurrences into the memory vector.
5.  **Save Results**: Store the outputs (final memory, group history, true partner list) to disk for the next notebook.

## 1.1. Setup & Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import pickle
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# Add src directory to path to import our modules
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.simulation import AgentSimulator
from src.affinity import compute_hd_embeddings, cosine_affinity
from src.grouping import build_adjacency, extract_groups
from src.binding import HDBinder
from src.memory import MemoryStore
from src.utils import set_seed

## 1.2. Set Hyperparameters

Here, we define the key parameters for our simulation run. These can be tuned to explore different dynamics.

In [ ]:
N = 100                  # Number of agents
D = 4000                 # Dimensionality of hypervectors
T = 5000                 # Total simulation time steps
SEED = 42                # Random seed for reproducibility

# Grouping parameters
TAU_INTRA = 1.2          # Multiplier for intra-group affinity
TAU_INTER = 0.8          # Multiplier for inter-group affinity
AFFINITY_THRESHOLD = 0.6 # Cosine similarity threshold for an edge

# Output directory
OUTPUT_DIR = '../results/notebook_run/'
os.makedirs(OUTPUT_DIR, exist_ok=True)

set_seed(SEED)

## 1.3. Instantiate Core Components

In [ ]:
simulator = AgentSimulator(N=N, seed=SEED)
binder = HDBinder(N=N, D=D, seed=SEED)
memory_store = MemoryStore(D=D)

# Random projection matrix for converting agent states to HD vectors
state_dim = simulator.state.shape[1] # e.g., 4 for (x, y, vx, vy)
W_proj = np.random.randn(D, state_dim)

print(f"Initialized simulation with {N} agents.")
print(f"HD vectors dimensionality: {D}")
print(f"State vector dimensionality: {state_dim}")

## 1.4. Run Simulation Loop

In [ ]:
group_history = []
state_history = []
prev_groups = np.zeros(N, dtype=int)

for t in tqdm(range(T), desc="Simulating"):
    # 1. Get current agent states
    states = simulator.step()
    state_history.append(states)
    
    # 2. Determine groups
    hd_embeddings = compute_hd_embeddings(states, W_proj)
    affinity_matrix = cosine_affinity(hd_embeddings, threshold=AFFINITY_THRESHOLD)
    adj_matrix = build_adjacency(affinity_matrix, prev_groups, TAU_INTRA, TAU_INTER)
    current_groups = extract_groups(adj_matrix)
    
    # 3. Bind co-occurring agents into memory
    unique_group_ids = np.unique(current_groups[current_groups != -1])
    for group_id in unique_group_ids:
        agent_indices = np.where(current_groups == group_id)[0].tolist()
        if len(agent_indices) > 1:
            binder.record_co_occurrence(memory_store, agent_indices)
            
    # 4. Periodic normalization of the memory vector
    if t > 0 and t % 200 == 0:
        memory_store.normalize()
    
    # 5. Store history and update previous groups
    group_history.append(current_groups)
    prev_groups = current_groups

# Final normalization
memory_store.normalize()

group_history = np.array(group_history)
state_history = np.array(state_history)
final_memory = memory_store.get_memory()
id_vectors = binder.E_id

print("Simulation complete.")

### Quick Visualization: Agent Trajectories

Let's plot the paths of the first few agents to see what the simulation looks like.

In [ ]:
plt.figure(figsize=(8, 8))
for i in range(min(N, 10)):
    plt.plot(state_history[:, i, 0], state_history[:, i, 1], alpha=0.7)
plt.title('Trajectories of First 10 Agents')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.grid(True)
plt.show()

## 1.5. Save Results

We now save the key artifacts to disk. These will be loaded by the second notebook for evaluation and visualization.

In [ ]:
# Generate the ground truth for evaluation
true_partners = {i: set() for i in range(N)}
for groups_at_t in group_history:
    for group_id in np.unique(groups_at_t[groups_at_t != -1]):
        members = np.where(groups_at_t == group_id)[0]
        for i1 in range(len(members)):
            for i2 in range(i1 + 1, len(members)):
                u, v = members[i1], members[i2]
                true_partners[u].add(v)
                true_partners[v].add(u)
true_partners = {k: list(v) for k, v in true_partners.items()}

# Package results into a dictionary
results = {
    'N': N,
    'D': D,
    'T': T,
    'seed': SEED,
    'final_memory': final_memory,
    'id_vectors': id_vectors,
    'group_history': group_history,
    'state_history': state_history,
    'true_partners': true_partners,
    'params': {
        'tau_intra': TAU_INTRA,
        'tau_inter': TAU_INTER,
        'affinity_threshold': AFFINITY_THRESHOLD
    }
}

# Save to file
output_path = os.path.join(OUTPUT_DIR, 'simulation_results.pkl')
with open(output_path, 'wb') as f:
    pickle.dump(results, f)

print(f"Results saved to: {output_path}")